##### **Databricks Bronze Layer:** Read .CSV from volume `raw` in `retail_sales_analytics_and_forecasting.bronze` schema

In [0]:
from pyspark.sql import SparkSession

#### **Define your single entity**

In [0]:
entity = "superstore"

#### **Static (batch) read to infer schema**

In [0]:
df_batch = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(f"/Volumes/retail_sales_analytics_and_forecasting/bronze/raw")
)

### **Clean column names**

In [0]:
def clean_column(col_name):
    return col_name.strip().lower().replace(" ", "_").replace("-", "_").replace("/", "_")

df_batch = df_batch.toDF(*[clean_column(c) for c in df_batch.columns])

#### **Capture the inferred schema**

In [0]:
schema_entity = df_batch.schema

#### **Now create a streaming DataFrame (structured streaming)**

In [0]:
df_stream = (
    spark.readStream.format("csv")
    .option("header", True)
    .schema(schema_entity)
    .load(f"/Volumes/retail_sales_analytics_and_forecasting/bronze/raw")
)

#### **Write to Bronze Delta Table (append mode, once trigger)**

In [0]:
(
    df_stream.writeStream.format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"/Volumes/retail_sales_analytics_and_forecasting/bronze/raw/{entity}")
    .trigger(once=True)
    .toTable(f"retail_sales_analytics_and_forecasting.bronze.{entity}")
)

In [0]:
print(f"✅ Bronze table created successfully: retail_sales_analytics_and_forecasting.bronze.{entity}")

✅ Bronze table created successfully: retail_sales_analytics_and_forecasting.bronze.superstore


In [0]:
%sql
SELECT COUNT(*) FROM retail_sales_analytics_and_forecasting.bronze.superstore;

COUNT(*)
9994


In [0]:
%sql
SELECT * FROM retail_sales_analytics_and_forecasting.bronze.superstore LIMIT 10;

row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031
5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164
6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.86,7,0,14.1694
7,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.28,4,0,1.9656
8,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.152,6,0.2,90.7152
9,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by Samsill,18.504,3,0.2,5.7825
10,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9,5,0,34.47
